# Text-to-SQL Model - Inference Only

This notebook loads the fine-tuned text-to-SQL model and provides an interface for generating SQL queries from natural language questions.


In [ ]:
import torch
import json
from unsloth import FastLanguageModel


## Load Fine-tuned Model

Loading the LoRA adapter merged with the base model.


In [ ]:
max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="../models/text-to-sql-lora",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)

# Enable fast inference mode
FastLanguageModel.for_inference(model)

print("✅ Model loaded successfully!")


## Inference Function

Function to generate SQL queries from natural language.


In [ ]:
def generate_sql(db_context, question, max_new_tokens=512, temperature=0.2, top_p=0.9):
    """
    Generate SQL query from database context and natural language question.
    
    Args:
        db_context (str): Database schema with CREATE TABLE statements
        question (str): Natural language question
        max_new_tokens (int): Maximum tokens to generate
        temperature (float): Sampling temperature (lower = more deterministic)
        top_p (float): Nucleus sampling parameter
    
    Returns:
        str: Generated SQL query
    """
    
    conversation = [
        {
            "role": "system",
            "content": "You are an expert SQL assistant. Generate accurate SQL queries based on the given database context and natural language questions."
        },
        {
            "role": "user",
            "content": f"Database Context:\n{db_context}\n\nQuestion: {question}"
        }
    ]
    
    prompt = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )
    
    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()
    return response

print("✅ Inference function ready!")


## SAP Table Schemas

Define complex SAP table structures for testing.


In [ ]:
# SAP ERP Table Schemas
SAP_SCHEMA = """
-- VBAK: Sales Document Header
CREATE TABLE VBAK (
    VBELN VARCHAR(10) PRIMARY KEY,  -- Sales Document Number
    ERDAT DATE,                      -- Creation Date
    AUART VARCHAR(4),                -- Sales Document Type
    NETWR DECIMAL(15,2),             -- Net Value
    WAERK VARCHAR(5),                -- Currency
    VKORG VARCHAR(4),                -- Sales Organization
    VTWEG VARCHAR(2),                -- Distribution Channel
    KUNNR VARCHAR(10),               -- Customer Number (FK to KNA1)
    BSTNK VARCHAR(35),               -- Customer PO Number
    ERZET TIME                       -- Creation Time
);

-- VBAP: Sales Document Item
CREATE TABLE VBAP (
    VBELN VARCHAR(10),               -- Sales Document Number (FK to VBAK)
    POSNR VARCHAR(6),                -- Item Number
    MATNR VARCHAR(18),               -- Material Number (FK to MARA)
    ARKTX VARCHAR(40),               -- Item Description
    KWMENG DECIMAL(15,3),            -- Order Quantity
    VRKME VARCHAR(3),                -- Sales Unit
    NETWR DECIMAL(15,2),             -- Net Value
    WAERK VARCHAR(5),                -- Currency
    WERKS VARCHAR(4),                -- Plant
    LGORT VARCHAR(4),                -- Storage Location
    PRIMARY KEY (VBELN, POSNR)
);

-- KNA1: Customer Master
CREATE TABLE KNA1 (
    KUNNR VARCHAR(10) PRIMARY KEY,   -- Customer Number
    NAME1 VARCHAR(35),               -- Customer Name
    LAND1 VARCHAR(3),                -- Country Key
    ORT01 VARCHAR(35),               -- City
    PSTLZ VARCHAR(10),               -- Postal Code
    STRAS VARCHAR(35),               -- Street
    KTOKD VARCHAR(4),                -- Customer Account Group
    BRSCH VARCHAR(4),                -- Industry
    ERDAT DATE,                      -- Creation Date
    STCEG VARCHAR(20)                -- VAT Registration Number
);

-- MARA: Material Master
CREATE TABLE MARA (
    MATNR VARCHAR(18) PRIMARY KEY,   -- Material Number
    MAKTX VARCHAR(40),               -- Material Description
    MTART VARCHAR(4),                -- Material Type
    MATKL VARCHAR(9),                -- Material Group
    MEINS VARCHAR(3),                -- Base Unit of Measure
    BRGEW DECIMAL(13,3),             -- Gross Weight
    NTGEW DECIMAL(13,3),             -- Net Weight
    GEWEI VARCHAR(3),                -- Weight Unit
    VOLUM DECIMAL(13,3),             -- Volume
    VOLEH VARCHAR(3),                -- Volume Unit
    ERDAT DATE,                      -- Creation Date
    ERSDA VARCHAR(8)                 -- Created By
);
"""

print("✅ SAP schemas loaded!")


## Test Cases

Run inference on complex SAP queries.


In [ ]:
test_cases = [
    "Find all sales orders with their customer names and total order values for customers in Germany (LAND1='DE')",
    "Get the top 10 materials by total sales quantity across all sales orders",
    "List all sales orders from 2023 with customer name, material description, quantity, and net value where order value exceeds 100000"
]

print("🔍 Testing Fine-tuned Text-to-SQL Model\n" + "="*80 + "\n")

for question in test_cases:
    print(f"QUESTION: {question}")
    sql_query = generate_sql(SAP_SCHEMA, question)
    print(f"\nGENERATED SQL:\n{sql_query}")
    print("\n" + "-"*80 + "\n")


## Gradio Interface

Interactive interface for text-to-SQL generation.


In [ ]:
import gradio as gr

def sql_interface(db_context, question, temperature, max_tokens):
    """
    Gradio interface wrapper for the text-to-SQL model.
    """
    if not question.strip():
        return "⚠️ Please enter a question."
    
    if not db_context.strip():
        return "⚠️ Please provide database context."
    
    # Generate SQL
    sql_query = generate_sql(
        db_context, 
        question,
        max_new_tokens=int(max_tokens),
        temperature=float(temperature)
    )
    
    return sql_query

# Create Gradio interface
with gr.Blocks(theme=gr.themes.Soft(), title="🗄️ Text-to-SQL") as demo:
    gr.Markdown("""
    # 🗄️ Text-to-SQL Generator
    
    ### Generate SQL queries from natural language questions.
    
    Provide database schema and ask questions in plain English!
    """)
    
    with gr.Row():
        with gr.Column(scale=2):
            # Database context input
            db_context_input = gr.Textbox(
                label="📋 Database Schema",
                placeholder="Enter CREATE TABLE statements...",
                lines=10,
                value=SAP_SCHEMA
            )
            
            # Question input
            question_input = gr.Textbox(
                label="❓ Your Question",
                placeholder="Ask a question about the data (e.g., 'Show all customers from Germany')",
                lines=3
            )
            
            # Advanced settings
            with gr.Accordion("⚙️ Advanced Settings", open=False):
                temperature_slider = gr.Slider(
                    minimum=0.1,
                    maximum=1.0,
                    value=0.2,
                    step=0.1,
                    label="Temperature (lower = more deterministic)",
                )
                max_tokens_slider = gr.Slider(
                    minimum=128,
                    maximum=1024,
                    value=512,
                    step=64,
                    label="Max Tokens",
                )
            
            # Submit button
            submit_btn = gr.Button("🚀 Generate SQL", variant="primary", size="lg")
        
        with gr.Column(scale=3):
            # SQL output
            sql_output = gr.Code(
                label="📝 Generated SQL Query",
                language="sql",
                lines=15
            )
    
    # Complex SAP examples
    gr.Examples(
        examples=[
            [
                SAP_SCHEMA,
                "Find all sales orders with their customer names and total order values for customers in Germany (LAND1='DE')",
                0.2,
                512
            ],
            [
                SAP_SCHEMA,
                "Get the top 10 materials by total sales quantity across all sales orders, including material description and total net value",
                0.2,
                512
            ],
            [
                SAP_SCHEMA,
                "List all sales orders from 2023 with customer name, city, material description, quantity, and net value where order value exceeds 100000",
                0.2,
                512
            ],
            [
                SAP_SCHEMA,
                "Show sales performance by sales organization (VKORG) with total number of orders, total net value, and average order value",
                0.2,
                512
            ],
            [
                SAP_SCHEMA,
                "Find customers who have ordered more than 5 different materials, showing customer name, country, and count of unique materials",
                0.2,
                512
            ],
            [
                SAP_SCHEMA,
                "Get all sales orders with customer and material details where the material type (MTART) is 'FERT' (finished goods) and order was created in the last 30 days",
                0.2,
                512
            ],
            [
                SAP_SCHEMA,
                "Show a summary of sales by material group (MATKL) and country, including total quantity sold, total net value, and number of unique customers",
                0.2,
                512
            ],
            [
                SAP_SCHEMA,
                "Find all sales order items where the ordered quantity exceeds the average quantity for that material across all orders",
                0.2,
                512
            ],
        ],
        inputs=[db_context_input, question_input, temperature_slider, max_tokens_slider],
        label="📚 Complex SAP Query Examples"
    )
    
    # Connect the interface
    submit_btn.click(
        fn=sql_interface,
        inputs=[db_context_input, question_input, temperature_slider, max_tokens_slider],
        outputs=[sql_output]
    )
    
    # Also allow Enter key to submit
    question_input.submit(
        fn=sql_interface,
        inputs=[db_context_input, question_input, temperature_slider, max_tokens_slider],
        outputs=[sql_output]
    )

print("✅ Gradio interface ready!")
print("Run: demo.launch() to start the interface")


## 🚀 Launch the Interface

Run this cell to start the interactive Gradio interface.


In [ ]:
# Launch the Gradio interface
# Use share=True to get a public link for 72 hours
# Use share=False to run locally only

demo.launch(
    share=False,
    server_name="0.0.0.0",
    server_port=7861,
    show_error=True,
)
